In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Quora Question Pairs

# Business Problem 

1. Description : 
Quora is a place to gain and share knowledge—about anything. It’s a platform to ask questions and connect with people who contribute unique insights and quality answers. This empowers people to learn from each other and to better understand the world.

Over 100 million people visit Quora every month, so it's no surprise that many people ask similarly worded questions. Multiple questions with the same intent can cause seekers to spend more time finding the best answer to their question, and make writers feel they need to answer multiple versions of the same question. Quora values canonical questions because they provide a better experience to active seekers and writers, and offer more value to both of these groups in the long term.

2. Problem Statement :
     * Identify Which question are asked on Quora are duplicates of questions
     * This could be useful to instantly provide answer to question that have already been asked.
     * We are tasked with predciting whether a pair of quesiton are duplicates or not.


# Useful link.
* Discussion :https://www.kaggle.com/anokas/data-analysis-xgboost-starter-0-35460-lb/notebook
* Kaggle Winning Solution : https://www.dropbox.com/sh/93968nfnrzh8bp5/AACZdtsApc1QSTQc7X0H3QZ5a?dl=0
* Blog 1 : https://engineering.quora.com/Semantic-Question-Matching-with-Deep-Learning
* Blog 2 : https://towardsdatascience.com/identifying-duplicate-questions-on-quora-top-12-on-kaggle-4c1cf93f1c30

# Real World /Business Objective and Constraints
1. The cost of misclassification can be very high.
2. We would want a probability of a pair of question to be duplicates so that we can choose any threshold of choice.
3. No strict latency concerns.
4. Interpretabilty is partial important.

# Machine Learning Problem
## Data 
### *Data overview:*
*  Data will be in a file Train.csv
* Train.csv Contains 5 columns : qid1, qid2, question1, question2, is_duplicate 
* size of train.csv: 60MB
* Number of rows in Train.csv = 404,290

### *Example Data point*

"id","qid1","qid2","question1","question2","is_duplicate"

"0","1","2","What is the step by step guide to invest in share market in india?","What is the step by step guide to invest in share market?","0"

"1","3","4","What is the story of Kohinoor (Koh-i-Noor) Diamond?","What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?","0"

"7","15","16","How can I be a good geologist?","What should I do to be a great geologist?","1"

"11","23","24","How do I read and find my YouTube comments?","How can I see all my Youtube comments?","1"


# Mapping the real world problem to an ML problem
 ## Type of Machine Learning Problem :
 * It is a binary Classification problem ,for a given pair of question we need to predict if they are duplicates or not
## Performance metrics: 
    * Source: https://www.kaggle.com/c/quora-question-pairs#evaluation

    * Metric(s):
        * log-loss : https://www.kaggle.com/wiki/LogarithmicLoss
        * Binary Confusion Matrix


 

# train and test Construction 
We build train and test by randomly splitting into 70:30 or 80:20 whatever we choose as we have sufficient points to worl out.

# Exploratoty Data Analysis



In [ ]:
pip install bs4


In [ ]:
pip install distance

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from subprocess import check_output
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import os
import gc 

import re
from nltk.corpus import stopwords
import distance
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup

# Reading data and basics stats

In [ ]:
df =pd.read_csv('/kaggle/input/quora-question-pairs/train.csv.zip')
print("Number of data points :",df.shape)

In [ ]:
df.head()

In [ ]:
df.info()

## We have got minimal number of data fields here , consisting of:
* id: Looks like a simple rodID
* qid{1,2}: The unique Id of each question in the pair
* question{1,2}: The actual textual content of question
* is_duplicate: The label that we are trying to predict -wether the two question are duplicates of each other.

# Distribution of data points among output classes
* Number of duplicates (similiar) and non-duplicates(non-similiar)question.

In [ ]:
sns.countplot(data=df,x='is_duplicate')

In [ ]:
print('~> Total number of question pairs for training:\n {}'.format(len(df)))

In [ ]:
#df['is_duplicate'] contains values of 0 and 1 only,
#so if we take a mean of those values that will give the percentage of 1s present.
print("Question pair are not similiar (is_duplicates=0):\n {}%".format(100-round(df['is_duplicate'].mean()*100,2)))
print("Question pair are similiar (is_duplicates=1):\n {}%".format(round(df['is_duplicate'].mean()*100,2)))

In [ ]:
# Number of unique Solution
qids=pd.Series(df['qid1'].tolist() + df['qid2'].tolist())
unique_qs=len(np.unique(qids))
qs_morethan_onetime=np.sum(qids.value_counts()>1)
print ('Total number of  Unique Questions are: {}\n'.format(unique_qs))
#print len(np.unique(qids))
print ('Number of unique questions that appear more than one time: {} ({}%)\n'.format(qs_morethan_onetime,qs_morethan_onetime/unique_qs*100))

print('Max number of times a single question is repeated: {}\n'.format(max(qids.value_counts()))) 
q_vals=qids.value_counts()
q_vals=q_vals.values

In [ ]:
x = ["unique_questions" , "Repeated Questions"]
y =  [unique_qs , qs_morethan_onetime]
plt.figure(figsize=(10, 6))
plt.title ("Plot representing unique and repeated questions  ")
sns.barplot(x,y)
plt.show()

# Checking for Duplicates

In [ ]:
#Checking whether there are any repeated pair of question.
pair_duplicates=df[['qid1','qid2','is_duplicate']].groupby(['qid1','qid2']).count().reset_index()
print('Number of duplicats ',pair_duplicates.shape[0]-df.shape[0])

In [ ]:
pair_duplicates=df[['qid1','qid2','is_duplicate']].groupby(['qid1','qid2']).count().reset_index()

# Number of occurrences of each question

In [ ]:
plt.figure(figsize=(20,10))
plt.hist(qids.value_counts(),bins=160)
plt.yscale('log',nonposy='clip')
plt.title('Log-Histogram of question appearance counts')
plt.xlabel('Number of occurences of question')
plt.ylabel('Number of questions')
print('Maximum number of times a single question is repeated : {} \n'.format(max(qids.value_counts())))

# Checking NULL values


In [ ]:
# Checking whether there are any rows with null value
nan_rows=df[df.isnull().any(1)]
print(nan_rows)

* There are two rows with null values in question2 

In [ ]:
# Filling the null valus with ' '(space)
df=df.fillna('')
nan_rows=df[df.isnull().any(1)]
print(nan_rows)


# BAsic Feature Extraction(before cleaning)

Constucting few features
* freq_qid1= Frequency of qid1's
* freq_qid2= Frequency of qid2's
* q1_len =Length of Question 1
* q2_len =Length of Question 2
* q1_n_words=Number of words in question 1
* q2_n_words=Number of words in question 2
* word_common=(Number of common unique words in Question 1 and Question 2)
* Word_Total=(Total number of words in Question1 + Total number of words in Question 2)
* Word_share=(word_common)/(word_tota)
* freq_q1+freq_q2=sum of frequency of qid1 and  qid2
* fred_q1-freq_q2=absolute diffrence of frequency of qid1 and qid2

In [ ]:
if os.path.isfile('df_fe_without_preprocessing_train.csv'):
    df=pd.read_csv('df_fe_without_preprocessing_train.csv',encoding='latin-1')
else:
    #how many times a question is repeated we can cross check ex: len(df[df['qid1']==3])=4
    df['freq_qid1']=df.groupby('qid1')['qid1'].transform('count') 
    df['freq_qid2']=df.groupby('qid2')['qid2'].transform('count')
    
    df['q1len']=df['question1'].str.len() #it will calculate string of each sentence
    df['q2len']=df['question2'].str.len()
    
    df['q1_n_words']=df['question1'].apply(lambda row : len(row.split(" ")))
    #it will calculate words in each sentence
    df['q2_n_words']=df['question2'].apply(lambda row : len(row.split(" ")))
    #it will calculate words in each sentence
    
    def normalized_word_common(row):
        w1=set(map(lambda word: word.lower().strip(),row['question1'].split(" ")))
        w2=set(map(lambda word: word.lower().strip(),row['question2'].split(" ")))
        return 1.0 * len(w1&w2)
    df['word_common']=df.apply(normalized_word_common,axis=1)
    
    def normalized_word_total(row):
        w1=set(map(lambda word: word.lower().strip(),row['question1'].split(" ")))
        w2=set(map(lambda word: word.lower().strip(),row['question2'].split(" ")))
        return 1.0 * (len(w1)+len(w2))
    df['word_total']=df.apply(normalized_word_total,axis=1)

    def normalized_word_share(row):
        w1=set(map(lambda word: word.lower().strip(),row['question1'].split(" ")))
        w2=set(map(lambda word: word.lower().strip(),row['question2'].split(" ")))
        return 1.0 * len(w1&w2)/(len(w1)+len(w2))
    df['word_share']=df.apply(normalized_word_share,axis=1)   
    
    df['freq_q1+q2']=df['freq_qid1']+df['freq_qid2']
    df['freq_q1-q2']=abs(df['freq_qid1']-df['freq_qid2'])
    
    df.to_csv("df_fe_without_preprocessing_train.csv",index=False)
df.head()

# Analysis of Some of the feature extracted 

* here are some question have only one single words 

In [ ]:
print('Minimum length of the question in question1 :',min(df['q1_n_words']))
print('Minimum length of the question in question2 :',min(df['q2_n_words']))

print('Number of Question with minimum length [question1]:',df[df['q1_n_words']==1].shape[0])
print('Number of Question with minimum length [question12]:',df[df['q2_n_words']==1].shape[0])

# Feature: word Share

In [ ]:
plt.figure(figsize=(12,8))

plt.subplot(1,2,1)
sns.violinplot(x='is_duplicate',y='word_share',data=df[0:])

plt.subplot(1,2,2)
sns.distplot(df[df['is_duplicate']==1]['word_share'][0:],label='1',color='orange')
sns.distplot(df[df['is_duplicate']==0]['word_share'][0:],label='0',color='blue')
plt.show()


* The distibution for normalized word_share have some overlap on the far right-hand side,i.e there are quite of question with high word similarity
* The average word share and Common no. of words of qid1 and qid2 is more when they are duplicate(Similar)

# Feature : Word_common

In [ ]:
plt.figure(figsize=(12,8))

plt.subplot(1,2,1)
sns.violinplot(x='is_duplicate',y='word_common',data=df[0:])

plt.subplot(1,2,2)
sns.distplot(df[df['is_duplicate']==1]['word_common'][0:],label='1',color='orange')
sns.distplot(df[df['is_duplicate']==0]['word_common'][0:],label='0',color='blue')
plt.show()


# <p> The distributions of the word_Common feature in similar and non-similar questions are highly overlapping </p>

# EDA: Advanced Feature Extraction.

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from subprocess import check_output
%matplotlib inline
import plotly.offline as py
import plotly.tools as tls
import os
import gc

import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz
from sklearn.manifold import TSNE
# Import the Required lib packages for WORD-Cloud generation
# https://stackoverflow.com/questions/45625434/how-to-install-wordcloud-in-python3-6
from wordcloud import WordCloud ,STOPWORDS
from os import path
from PIL import Image
import distance



In [ ]:
#https://stackoverflow.com/questions/12468179/unicodedecodeerror-utf8-codec-cant-decode-byte-0x9c
if os.path.isfile("df_fe_without_preprocessing_train.csv"):
    df=pd.read_csv('df_fe_without_preprocessing_train.csv',encoding='latin-1')
    df = df.fillna("")
    df.head()
else:
    print('get df_fe_without_preprocessing_train.csv from drive or run the previous notebooks')
    

In [ ]:
df.head(2)

# Prepricessing of Text.
* Preprocessing
    * Removing the HTML TAGS
    * Removing Punctuation
    * Performing Stemming
    * Removing Stopwords 
    * Expanding Contradiction etc .

In [ ]:
#for getting result in 4 decimal points
Safe_div=0.0001
Stop_words=stopwords.words('english')
def preprocess(x):
    x=str(x).lower()
    x=x.replace(",000,000", "m").replace(",000", "k").replace("′", "'").replace("’", "'")\
                           .replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
                           .replace("n't", " not").replace("what's", "what is").replace("it's", "it is")\
                           .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
                           .replace("he's", "he is").replace("she's", "she is").replace("'s", " own")\
                           .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")\
                           .replace("€", " euro ").replace("'ll", " will")
    
    x = re.sub(r"([0-9]+)000000", r"\1m", x)
    x = re.sub(r"([0-9]+)000", r"\1k", x)
    
    stemming =PorterStemmer()
    pattern=re.compile('\W')
    
    if type(x)==type(''):
        x=re.sub(pattern,' ',x)
    if type(x)==type(''):
        x=stemming.stem(x)
        exam=BeautifulSoup(x)
        x=exam.get_text()
    
    return x


* Function to compute and get the feature : With 2 parameter of Question1 and Question2

# Advance Feature Extraction (NLP and Fuzzy Features)

* Defination
    * Token: You get a token by splitting sentence a space
    * Stop_words: Stop words as per NLTK.
    * Word: A Token that is not a stop_word 

* Feature :
    * cwc_min: Ratio of common_words_count to min length of word count of Q1 and Q2
           cwc_min =common_word_count/(min(len(q1_words),len(q2_words))
    * cwc_max: Ratio of common_words_count to max length of word count of Q1 and Q2
           cwc_max =common_word_count/(max(len(q1_words),len(q2_words)) 
    
    * csc_min: Ratio of common_stop_count to min length of stops count of Q1 and Q2
           csc_min =common_stop_count/(min(len(q1_stops),len(q2_stops))
    * csc_max: Ratio of common_stop_count to max length of stops count of Q1 and Q2
           csc_max =common_stop_count/(max(len(q1_stops),len(q2_stops))
    
    * ctc_min: Ratio of common_token_count to min length of token count of Q1 and Q2
           ctc_min =common_token_count/(min(len(q1_tokens),len(q2_tokens))
    * ctc_max: Ratio of common_token_count to max length of token count of Q1 and Q2
           ctc_max =common_token_count/(max(len(q1_tokens),len(q2_tokens))      
           
    * last_words_eq: Check if Last word of both question is equal or not 
           last_words_eq=int(q1_token[-1]==q2_token[-1])
    * first_words_eq: Check if First word of both question is equal or not 
           first_words_eq=int(q1_token[-1]==q2_token[-1])
           
    * abs_len_diff: Abs length differnce 
           abs_len_diff=abs(len(q1_tokens)-len(q2_token))
           
    * mean_len: Average Token Lenght of both questions
           mean_len=((len(q1_tokens)+len(q2_tokens))/2 )
           
    * fuzz_ratio: https://github.com/seatgeek/fuzzywuzzy#usage http://chairnerd.seatgeek.com/fuzzywuzzy-fuzzy-string-matching-in-python/


    * fuzz_partial_ratio:
             https://github.com/seatgeek/fuzzywuzzy#usage http://chairnerd.seatgeek.com/fuzzywuzzy-fuzzy-string-matching-in-python/


    *  token_sort_ratio: https://github.com/seatgeek/fuzzywuzzy#usagehttp://chairnerd.seatgeek.com/fuzzywuzzy-fuzzy-string-matching-in-python/

    * token_set_ratio: https://github.com/seatgeek/fuzzywuzzy#usage http://chairnerd.seatgeek.com/fuzzywuzzy-fuzzy-string-matching-in-python/

    * longest_substr_ratio: Ratio of length longest common substring to min length of token count Q1 and Q2. 
           longest_substr_ratio= len(longest common string)/(min(len(q1_tokens),len(q2_tokens))

In [ ]:
def get_token_features(q1, q2):
    token_features = [0.0]*10
    
    # Converting the Sentence into Tokens: 
    q1_tokens = q1.split()
    q2_tokens = q2.split()

    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return token_features
    # Get the non-stopwords in Questions
    q1_words = set([word for word in q1_tokens if word not in Stop_words])
    q2_words = set([word for word in q2_tokens if word not in Stop_words])
    
    #Get the stopwords in Questions
    q1_stops = set([word for word in q1_tokens if word in Stop_words])
    q2_stops = set([word for word in q2_tokens if word in Stop_words])
    
    # Get the common non-stopwords from Question pair
    common_word_count = len(q1_words.intersection(q2_words))
    
    # Get the common stopwords from Question pair
    common_stop_count = len(q1_stops.intersection(q2_stops))
    
    # Get the common Tokens from Question pair
    common_token_count = len(set(q1_tokens).intersection(set(q2_tokens)))
    
    
    token_features[0] = common_word_count / (min(len(q1_words), len(q2_words)) + Safe_div)
    token_features[1] = common_word_count / (max(len(q1_words), len(q2_words)) + Safe_div)
    token_features[2] = common_stop_count / (min(len(q1_stops), len(q2_stops)) + Safe_div)
    token_features[3] = common_stop_count / (max(len(q1_stops), len(q2_stops)) + Safe_div)
    token_features[4] = common_token_count / (min(len(q1_tokens), len(q2_tokens)) + Safe_div)
    token_features[5] = common_token_count / (max(len(q1_tokens), len(q2_tokens)) + Safe_div)
    
    # Last word of both question is same or not
    token_features[6] = int(q1_tokens[-1] == q2_tokens[-1])
    
    # First word of both question is same or not
    token_features[7] = int(q1_tokens[0] == q2_tokens[0])
    
    token_features[8] = abs(len(q1_tokens) - len(q2_tokens))
    
    #Average Token Length of both Questions
    token_features[9] = (len(q1_tokens) + len(q2_tokens))/2
    return token_features

# get the Longest Common sub string

def get_longest_substr_ratio(a, b):
    strs = list(distance.lcsubstrings(a, b))
    if len(strs) == 0:
        return 0
    else:
        return len(strs[0]) / (min(len(a), len(b)) + 1)

def extract_features(df):
    # preprocessing each question
    df["question1"] = df["question1"].fillna("").apply(preprocess)
    df["question2"] = df["question2"].fillna("").apply(preprocess)

    print("token features...")
    
    # Merging Features with dataset
    
    token_features = df.apply(lambda x: get_token_features(x["question1"], x["question2"]), axis=1)
    
    df["cwc_min"]       = list(map(lambda x: x[0], token_features))
    df["cwc_max"]       = list(map(lambda x: x[1], token_features))
    df["csc_min"]       = list(map(lambda x: x[2], token_features))
    df["csc_max"]       = list(map(lambda x: x[3], token_features))
    df["ctc_min"]       = list(map(lambda x: x[4], token_features))
    df["ctc_max"]       = list(map(lambda x: x[5], token_features))
    df["last_word_eq"]  = list(map(lambda x: x[6], token_features))
    df["first_word_eq"] = list(map(lambda x: x[7], token_features))
    df["abs_len_diff"]  = list(map(lambda x: x[8], token_features))
    df["mean_len"]      = list(map(lambda x: x[9], token_features))
    ("""#1) list is used to typecast the map object. 
    #Map function, after applying the given function on the iterable returns a map object,
    #which is then typecasted to list.
    #2) token_features is a list of lists. Each of the sub list contains 5 elements. 
    And if you do token_features[0], you will get the first sub list of the whole list.
    Which means that you will get the first list which contains 5 elements.
    But here we want the first element of all the sublists as one column that is 
    why we used map function. I hope you got the point""")
   
    #Computing Fuzzy Features and Merging with Dataset
    
    # do read this blog: http://chairnerd.seatgeek.com/fuzzywuzzy-fuzzy-string-matching-in-python/
    # https://stackoverflow.com/questions/31806695/when-to-use-which-fuzz-function-to-compare-2-strings
    # https://github.com/seatgeek/fuzzywuzzy
    print("fuzzy features..")

    df["token_set_ratio"]       = df.apply(lambda x: fuzz.token_set_ratio(x["question1"], x["question2"]), axis=1)
    # The token sort approach involves tokenizing the string in question, sorting the tokens alphabetically, and 
    # then joining them back into a string We then compare the transformed strings with a simple ratio().
    df["token_sort_ratio"]      = df.apply(lambda x: fuzz.token_sort_ratio(x["question1"], x["question2"]), axis=1)
    df["fuzz_ratio"]            = df.apply(lambda x: fuzz.QRatio(x["question1"], x["question2"]), axis=1)
    df["fuzz_partial_ratio"]    = df.apply(lambda x: fuzz.partial_ratio(x["question1"], x["question2"]), axis=1)
    df["longest_substr_ratio"]  = df.apply(lambda x: get_longest_substr_ratio(x["question1"], x["question2"]), axis=1)
    return df

In [ ]:
if os.path.isfile('nlp_features_train.csv'):
    df = pd.read_csv("nlp_features_train.csv",encoding='latin-1')
    df.fillna('')
else:
    print("Extracting features for train:")
    df = pd.read_csv("../input/quora-question-pairs/train.csv.zip")
    df = extract_features(df)
    df.to_csv("nlp_features_train.csv", index=False)
df.head(2)

# Analysing of extracted features

## Plotting Word Clouds
* Creating Word Cloud of Duplicates and Non-Duplicates Question Pairs
* We can observe the most frequent occuring words.

In [ ]:
df_dupicates = df[df['is_duplicate']==1]
df_nondupicates = df[df['is_duplicate']==0]

#Converting 2d array of q1 and q2 and Flatten the array : like {{1,2},{3,4}} to {1,2,3,4}
p=np.dstack([df_dupicates['question1'],df_dupicates['question2']]).flatten()
n=np.dstack([df_nondupicates['question1'],df_nondupicates['question2']]).flatten()

print ("Number of data points in class 1 (duplicate pairs) :",len(p))
print ("Number of data points in class 0 (non duplicate pairs) :",len(n))

#Saving the np array into a text file 
np.savetxt('train_p.txt',p,delimiter=' ',fmt='%s')
np.savetxt('train_n.txt',n,delimiter=' ',fmt='%s')

In [ ]:
d=path.dirname('.')

textp_w=open(path.join(d,'train_p.txt')).read()
textn_w=open(path.join(d,'train_n.txt')).read()
stopwords=set(Stop_words)
stopwords.add("said")
stopwords.add("br")
stopwords.add(" ")
stopwords.remove("not")
stopwords.remove("no")
#stopwords.remove("good")
#stopwords.remove("love")
#stopwords.remove("like")
#stopwords.remove("best")
#stopwords.remove("!")

print('total number of words in duplicates pair question:',len(textp_w))
print('total number of words in non duplicates pair question:',len(textn_w))

# Word Cloud generated from pair question's Text

In [ ]:
wc=WordCloud(background_color='white',max_words=len(textp_w),stopwords=Stop_words)
wc.generate(textp_w)
print("Word cloud for Duplicates Question pair")
plt.imshow(wc,interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
wc=WordCloud(background_color='white',max_words=len(textn_w),stopwords=Stop_words)
wc.generate(textn_w)
print("Word cloud for non Duplicates Question pair")
plt.imshow(wc,interpolation='bilinear')
plt.axis('off')
plt.show()

# Pair Plot of feature ['ctc_min','cwc_min','csc_min','token_sort_ratio']

In [ ]:
n=df.shape[0]
sns.pairplot(df[['ctc_min','cwc_min','csc_min','token_sort_ratio','is_duplicate']][0:n],hue='is_duplicate',vars=['ctc_min', 'cwc_min', 'csc_min', 'token_sort_ratio'])
plt.show()

In [ ]:
# Distribution of the token_sort_ratio
plt.figure(figsize=(10,8))
plt.subplot(1,2,1)
sns.violinplot(x='is_duplicate',y='token_sort_ratio',data=df[0:])
plt.subplot(1,2,2)
sns.distplot(df[df['is_duplicate']==1.0]['token_sort_ratio'][0:],label='1',color='red')
sns.distplot(df[df['is_duplicate']==0]['token_sort_ratio'][0:],label='0',color='blue')
plt.show()

In [ ]:
# Distribution of the token_sort_ratio
plt.figure(figsize=(10,8))
plt.subplot(1,2,1)
sns.violinplot(x='is_duplicate',y='fuzz_ratio',data=df[0:])
plt.subplot(1,2,2)
sns.distplot(df[df['is_duplicate']==1.0]['fuzz_ratio'][0:],label='1',color='red')
sns.distplot(df[df['is_duplicate']==0]['fuzz_ratio'][0:],label='0',color='blue')
plt.show()

In [ ]:
# Using Tsne for Dimensionality reduction for 15 features (Generated after cleaning the data) to 3 dimension
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler
dfp_subsampled=df[0:5000] # picking 5000 data points
X=MinMaxScaler().fit_transform(dfp_subsampled[['cwc_min', 'cwc_max', 'csc_min', 'csc_max' , 'ctc_min' , 'ctc_max' , 'last_word_eq', 'first_word_eq' , 'abs_len_diff' , 'mean_len' , 'token_set_ratio' , 'token_sort_ratio' ,  'fuzz_ratio' , 'fuzz_partial_ratio' , 'longest_substr_ratio']])
y=dfp_subsampled['is_duplicate'].values

In [ ]:
tsne2d=TSNE(n_components=2,
            init='random',
            random_state=101,
            method='barnes_hut',
            n_iter=1000,
            verbose=2,
            angle=0.5).fit_transform(X)

In [ ]:
df=pd.DataFrame({'x':tsne2d[:,0],'y':tsne2d[:,1],'label':y})
#draw the plot in appropriate place in the grid
sns.lmplot(data=df,x='x',y='y',hue='label',fit_reg=False,size=8,palette='Set1',markers=['s','o'])
plt.title("perplexity : {} and max_iter : {}".format(30, 1000))
plt.show()

In [ ]:
tsne3d=TSNE(n_components=3,
            init='random',
            random_state=101,
            method='barnes_hut',
            n_iter=1000,
            verbose=2,
            angle=0.5).fit_transform(X)

In [ ]:
tracel =go.Scatter3d(x=tsne3d[:,0]
                     ,y=tsne3d[:,1],
                     z=tsne3d[0:,2],
                     mode='markers',
                     marker=dict(sizemode='diameter',
                                 color = y,
                                 colorscale = 'Portland',
                                 colorbar = dict(title = 'duplicate'),
                                 line=dict(color='rgb(255, 255, 255)'),
                                 opacity=0.75)
                    )
data=[tracel]
layout=dict(height=800,width=800,title='3d embedding with engineered features')
fig=dict(data=data,layout=layout)
py.iplot(fig,filename='3DBubble')

# Featurizing text data with tfidf weighted word-vectors

In [ ]:
import pandas as pd
import matplotlib.pyplot  as plt
import seaborn as sns
import numpy as np
import time 
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import warnings
warnings.filterwarnings('ignore')

import os
import re
import sys
from tqdm import tqdm
import spacy
# exctract word2vec vectors
# https://github.com/explosion/spaCy/issues/1721

In [ ]:
#avoiding decoding problems
df=pd.read_csv('../input/quora-question-pairs/train.csv.zip')
# encode question to unicode
# https://stackoverflow.com/a/6812069
df['question1']=df['question1'].apply(lambda x:str(x))
df['question2']=df['question2'].apply(lambda x:str(x))

In [ ]:
df.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# merging text
questions = list(df['question1'])+list(df['question2'])
tf_idf=TfidfVectorizer(lowercase=False)
tf_idf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2_tf_idf=dict(zip(tf_idf.get_feature_names(),tf_idf.idf_))

* After we find Tf-idf scores ,we convert each question to a weighted average of word2vec vectors by these scores 
* We are using a pre-trained **GLOVE** model which come free with "*Spacy*"(https://spacy.io/usage/vectors-similarity)
* It is trained on Wikipedia and therefore, it is stronger in terms of word semantics.

In [ ]:
pip install spacy 

In [ ]:
# "en_vectors_web_sm", which includes over 1 million unique vectors.
nlp = spacy.load('en_core_web_sm')
vecs1 = []
for q1 in tqdm(list(df['question1'])): # tqdm is used to print the progress bar
    doc1 = nlp(q1) 
    # 
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])
    for word1 in doc1:
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = wor2_tf_idf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
df['q1_feats_m'] = list(vecs1)

In [ ]:
vecs2 = []
for q2 in tqdm(list(df['question2'])): # tqdm is used to print the progress bar
    doc2 = nlp(q2) 
    # 
    mean_vec2 = np.zeros([len(doc2), len(doc2[0].vector)])
    for word2 in doc2:
        # word2vec
        vec2 = word2.vector
        # fetch df score
        try:
            idf = wor2_tf_idf[str(word2)]
        except:
            idf = 0
        # compute final vec
        mean_vec2 += vec2 * idf
    mean_vec2 = mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)
df['q1_feats_m'] = list(vecs2)


In [ ]:
##